In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [4]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

In [5]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [6]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [7]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [8]:
device = provider.get_backend('ibm_cairo')
device

<IBMQBackend('ibm_cairo') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [9]:
device.configuration().to_dict()['max_shots']

100000

In [10]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [11]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [12]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [13]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [14]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [15]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [16]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [17]:
#checking circuit

In [18]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [19]:
circuit = front_layer.copy()

In [20]:
apply_checking_circuit(circuit, [0], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [0], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1937118134.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1937118134.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [21]:
circuit.draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [22]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [23]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [24]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68324/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [25]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«     └┬──────────────┤├────────────┤
«q_3: ─┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«      └──────────────┘└────────────┘
«q_4: ───────────────────────────────
«

In [26]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [27]:
cuts = [(circuit.qubits[0], 3),(circuit.qubits[0], 4)]

In [28]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«      ┌──────────────┐                
«q_0: ─┤ U(π/2,0,π/2) ├────────────────
«      ├──────────────┤                
«q_1: ─┤ U(π/2,0,π/2) ├────────────────
«     ┌┴──────────────┤┌──────────────┐
«q_2: ┤ Ry(0.003620

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐ 
«q_0: ──┤ U(π/2,0,π) ├─
«       ├────────────┤ 
«q_1: ──┤ U(π/2,0,π) ├─
«     ┌─┴────────────┴┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_

           ┌───┐      ┌───────────────┐            ┌────────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
           ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
     ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
     ├───────────────┤                  │        │         │         »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
     └─────┬───┬─────┘                  │        │       ┌───┐       »
q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
           └───┘                                         └───┘       »
«       ┌────────────┐               
«q_0: ──┤ U(π/2,0,π) ├───────────────
«       ├────────────┤               
«q_1: ──┤ U(π/2,0,π) ├───────────────
«     ┌─┴────────────┴┐┌────────────┐
«q_2: ┤ Ry(0.0036205) ├┤ U(π/

In [29]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 1)), (1, Qubit(QuantumRegister(4, 'q'), 0)), (0, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}


In [30]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [31]:
total_variants

24

In [32]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [33]:
fragments_list[0][0].draw()

┌───┐                             ┌───┐                       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │  │       └───┘                       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────────────────────
     └───┘└───────────────┘    │ ┌────────────────┐┌──────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
                                 └────────────────┘└──────────────┘

In [34]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
     ┌───┐                             ┌───┐                       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────────────────────
     ├───┤┌───────────────┐ │  │       └───┘                       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────────────────────
     └───┘└───────────────┘    │ ┌────────────────┐┌──────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├
                                 └────────────────┘└──────────────┘

fragment 1:
                                                                 »
q_0: ───────────────────────────────────■────────────────────────»
     ┌───────────────┐                  │                        »
q_1: ┤ Ry(0.0038243) ├──────────────────┼──■───────────────────■─»
     └─────┬───┬─────┘┌───────────────┐ │  │ ┌───────────────┐ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─┼─»
     ┌─────┴───┴─────┐└───────────────┘      └───────────────┘ │ »
q_3: ┤ Ry(-0.019851) ├────────

In [35]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [36]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(3, 'q'), 1)),
  (1, Qubit(QuantumRegister(4, 'q'), 0)),
  (0, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((1, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((0, Qubit(QuantumRegister(3, 'q'), 0)),)}

In [37]:
num_fragments = len(fragments)
num_fragments

2

In [43]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [44]:
fragments[0].draw()

┌───┐                             ┌───┐       
q_0: ┤ H ├──────────────────■──■───────┤ H ├───────
     ├───┤┌───────────────┐ │  │       └───┘       
q_1: ┤ X ├┤ Ry(-0.003534) ├─■──┼───────────────────
     └───┘└───────────────┘    │ ┌────────────────┐
q_2: ──────────────────────────■─┤ Ry(-0.0036834) ├
                                 └────────────────┘

In [45]:
fragments[1].draw()

»
q_0: ───────────────────────────────────■────────────────────────»
     ┌───────────────┐                  │                        »
q_1: ┤ Ry(0.0038243) ├──────────────────┼──■───────────────────■─»
     └─────┬───┬─────┘┌───────────────┐ │  │ ┌───────────────┐ │ »
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─┤ Ry(-0.015751) ├─┼─»
     ┌─────┴───┴─────┐└───────────────┘      └───────────────┘ │ »
q_3: ┤ Ry(-0.019851) ├─────────────────────────────────────────■─»
     └───────────────┘                                           »
«                      
«q_0: ─────────────────
«     ┌───────────────┐
«q_1: ┤ Ry(0.0036205) ├
«     └───────────────┘
«q_2: ─────────────────
«      ┌──────────────┐
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘

In [46]:
hardware_index = 1

In [47]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, initial_layout = [23,25,24,26], opt_lvl = 3, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [48]:
#contains 25 basis
len(tomography_circuits)

25

In [49]:
#contains three fragments
len(tomography_circuits[0])

2

In [50]:
len(tomography_circuits[0][1])

1

In [51]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [52]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [53]:
multi_exp_circuits[1][-200].draw()

┌─────────────────┐ ░                                         »
  q_0: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────■────»
       └─────────────────┘ ░ ┌───────────────┐                  │    »
  q_1: ────────────────────░─┤ Ry(0.0038243) ├──────────────────┼──■─»
                           ░ └─────┬───┬─────┘┌───────────────┐ │  │ »
  q_2: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■──■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘      »
  q_3: ────────────────────░─┤ Ry(-0.019851) ├───────────────────────»
                           ░ └───────────────┘                       »
c17_0: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_1: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_2: ══════════════════════════════════════════════════════════════»
                                                                     »
c17_3: ══════════════════════════════════════════════════════════════»
                                                                     »
«                                                           ░ ┌───┐      ┌─┐
«  q_0: ────────────────────────────────────────────────────░─┤ H ├──────┤M├
«                           ┌───────────────┐┌────────────┐ ░ └┬─┬┘      └╥┘
«  q_1: ──────────────────■─┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├─░──┤M├────────╫─
«       ┌───────────────┐ │ └┬──────────────┤└────────────┘ ░  └╥┘ ┌─┐    ║ 
«  q_2: ┤ Ry(-0.015751) ├─┼──┤ U(π/2,0,π/2) ├───────────────░───╫──┤M├────╫─
«       └───────────────┘ │  ├──────────────┤┌────────────┐ ░   ║  └╥┘┌─┐ ║ 
«  q_3: ──────────────────■──┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░───╫───╫─┤M├─╫─
«                            └──────────────┘└────────────┘ ░   ║   ║ └╥┘ ║ 
«c17_0: ════════════════════════════════════════════════════════╬═══╬══╬══╩═
«                                                               ║   ║  ║    
«c17_1: ════════════════════════════════════════════════════════╩═══╬══╬════
«                                                                   ║  ║    
«c17_2: ════════════════════════════════════════════════════════════╩══╬════
«                                                                      ║    
«c17_3: ═══════════════════════════════════════════════════════════════╩════
«

In [55]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [23,25,24,26], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  25d66422-95d8-4904-9b6c-407e0a47dfcd
Job Status: job has successfully run
job_id:  6377111bb95e160aefc27866
Job Status: job is queued (124)   

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/6377111bb95e160aefc27866/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7fa26ade6eb0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [ ]:
frag_datas[0][0].results[0].header.name

In [ ]:
num_tomo_circuits

In [280]:
test = [[], []]

In [283]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [288]:
frag_datas[0][0].get_counts()

[{'111': 49821, '110': 50179},
 {'010': 24920, '110': 25020, '011': 24940, '111': 25120},
 {'001': 1, '011': 25054, '110': 24995, '010': 24777, '111': 25173},
 {'100': 33530, '111': 16596, '101': 33325, '110': 16549},
 {'101': 16480,
  '011': 8289,
  '111': 8501,
  '110': 8318,
  '001': 16834,
  '100': 16535,
  '000': 16633,
  '010': 8410},
 {'001': 16855,
  '100': 16727,
  '110': 8376,
  '101': 16309,
  '111': 8433,
  '011': 8356,
  '010': 8408,
  '000': 16536},
 {'111': 16674, '101': 33398, '110': 16714, '100': 33214},
 {'010': 8232,
  '000': 16747,
  '111': 8472,
  '011': 8352,
  '101': 16626,
  '001': 16589,
  '110': 8426,
  '100': 16556},
 {'111': 8339,
  '011': 8526,
  '101': 16552,
  '010': 8423,
  '000': 16565,
  '100': 16510,
  '001': 16718,
  '110': 8367},
 {'111': 16447, '100': 33480, '110': 16825, '101': 33248},
 {'001': 16470,
  '100': 16911,
  '110': 8396,
  '010': 8295,
  '000': 16689,
  '111': 8512,
  '011': 8085,
  '101': 16642},
 {'110': 8298,
  '100': 16600,
  '001':

In [290]:
frag_datas[0][0].get_counts()

[{'111': 49821, '110': 50179},
 {'010': 24920, '110': 25020, '011': 24940, '111': 25120},
 {'001': 1, '011': 25054, '110': 24995, '010': 24777, '111': 25173},
 {'100': 33530, '111': 16596, '101': 33325, '110': 16549},
 {'101': 16480,
  '011': 8289,
  '111': 8501,
  '110': 8318,
  '001': 16834,
  '100': 16535,
  '000': 16633,
  '010': 8410},
 {'001': 16855,
  '100': 16727,
  '110': 8376,
  '101': 16309,
  '111': 8433,
  '011': 8356,
  '010': 8408,
  '000': 16536},
 {'111': 16674, '101': 33398, '110': 16714, '100': 33214},
 {'010': 8232,
  '000': 16747,
  '111': 8472,
  '011': 8352,
  '101': 16626,
  '001': 16589,
  '110': 8426,
  '100': 16556},
 {'111': 8339,
  '011': 8526,
  '101': 16552,
  '010': 8423,
  '000': 16565,
  '100': 16510,
  '001': 16718,
  '110': 8367},
 {'111': 16447, '100': 33480, '110': 16825, '101': 33248},
 {'001': 16470,
  '100': 16911,
  '110': 8396,
  '010': 8295,
  '000': 16689,
  '111': 8512,
  '011': 8085,
  '101': 16642},
 {'110': 8298,
  '100': 16600,
  '001':

In [ ]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

In [304]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1


In [305]:
len(organized_data)

25

In [306]:
organized_data[2]

[{'11': {(('S0',), ('Zm',)): 1,
   (('S2',), ('Zp',)): 1,
   (('S3',), ('Xp',)): 2,
   (('S0',), ('Zp',)): 0,
   (('S0',), ('Xp',)): 0,
   (('S0',), ('Xm',)): 0,
   (('S0',), ('Yp',)): 0,
   (('S0',), ('Ym',)): 0,
   (('S1',), ('Zp',)): 0,
   (('S1',), ('Zm',)): 0,
   (('S1',), ('Xp',)): 0,
   (('S1',), ('Xm',)): 0,
   (('S1',), ('Yp',)): 0,
   (('S1',), ('Ym',)): 0,
   (('S2',), ('Zm',)): 0,
   (('S2',), ('Xp',)): 0,
   (('S2',), ('Xm',)): 0,
   (('S2',), ('Yp',)): 0,
   (('S2',), ('Ym',)): 0,
   (('S3',), ('Zp',)): 0,
   (('S3',), ('Zm',)): 0,
   (('S3',), ('Xm',)): 0,
   (('S3',), ('Yp',)): 0,
   (('S3',), ('Ym',)): 0},
  '10': {(('S0',), ('Zm',)): 99999,
   (('S0',), ('Xp',)): 50450,
   (('S0',), ('Xm',)): 49550,
   (('S0',), ('Ym',)): 49985,
   (('S0',), ('Yp',)): 50014,
   (('S1',), ('Zm',)): 33394,
   (('S1',), ('Xm',)): 16777,
   (('S1',), ('Xp',)): 16752,
   (('S1',), ('Ym',)): 16625,
   (('S1',), ('Yp',)): 16514,
   (('S2',), ('Zm',)): 33457,
   (('S2',), ('Xp',)): 16631,
   

In [307]:
organized_data[-1][-1]

{'100': {(('S0',), ('Zm',)): 1,
  (('S0',), ('Zp',)): 28,
  (('S0',), ('Xm',)): 14,
  (('S0',), ('Xp',)): 17,
  (('S0',), ('Ym',)): 19,
  (('S0',), ('Yp',)): 14,
  (('S1',), ('Zp',)): 11,
  (('S1',), ('Zm',)): 16,
  (('S1',), ('Xm',)): 39,
  (('S1',), ('Ym',)): 34,
  (('S1',), ('Yp',)): 15,
  (('S2',), ('Zp',)): 8,
  (('S2',), ('Zm',)): 21,
  (('S2',), ('Xm',)): 3,
  (('S2',), ('Xp',)): 21,
  (('S2',), ('Ym',)): 3,
  (('S2',), ('Yp',)): 28,
  (('S3',), ('Zm',)): 15,
  (('S3',), ('Zp',)): 10,
  (('S3',), ('Xm',)): 8,
  (('S3',), ('Xp',)): 20,
  (('S3',), ('Yp',)): 1,
  (('S3',), ('Ym',)): 17,
  (('S1',), ('Xp',)): 0},
 '101': {(('S0',), ('Zp',)): 1,
  (('S0',), ('Ym',)): 1,
  (('S1',), ('Zp',)): 1,
  (('S2',), ('Zm',)): 1,
  (('S0',), ('Zm',)): 0,
  (('S0',), ('Xp',)): 0,
  (('S0',), ('Xm',)): 0,
  (('S0',), ('Yp',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S1',), ('Xp',)): 0,
  (('S1',), ('Xm',)): 0,
  (('S1',), ('Yp',)): 0,
  (('S1',), ('Ym',)): 0,
  (('S2',), ('Zp',)): 0,
  (('S2',), ('Xp',

In [308]:
recombined_lists = []
filtered_recombined_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))

{'11': {(('S0',), ('Zm',)): 49821, (('S0',), ('Xp',)): 24940, (('S0',), ('Xm',)): 25120, (('S0',), ('Yp',)): 25054, (('S0',), ('Ym',)): 25173, (('S1',), ('Zm',)): 16596, (('S1',), ('Xp',)): 8289, (('S1',), ('Xm',)): 8501, (('S1',), ('Ym',)): 8433, (('S1',), ('Yp',)): 8356, (('S2',), ('Zm',)): 16674, (('S2',), ('Xm',)): 8472, (('S2',), ('Xp',)): 8352, (('S2',), ('Ym',)): 8339, (('S2',), ('Yp',)): 8526, (('S3',), ('Zm',)): 16447, (('S3',), ('Xm',)): 8512, (('S3',), ('Xp',)): 8085, (('S3',), ('Yp',)): 8416, (('S3',), ('Ym',)): 8163, (('S0',), ('Zp',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '10': {(('S0',), ('Zm',)): 50179, (('S0',), ('Xp',)): 24920, (('S0',), ('Xm',)): 25020, (('S0',), ('Ym',)): 24995, (('S0',), ('Yp',)): 24777, (('S1',), ('Zm',)): 16549, (('S1',), ('Xm',)): 8318, (('S1',), ('Xp',)): 8410, (('S1',), ('Ym',)): 8376, (('S1',), ('Yp',)): 8408, (('S2',), ('Zm',)): 16714, (('S2',), ('Xp',)): 8232, (('S2',), ('Xm',)): 8426, (('S2',), ('Yp',)):

{'00': {(('S0',), ('Zp',)): 1, (('S1',), ('Zm',)): 33171, (('S1',), ('Xm',)): 16878, (('S1',), ('Xp',)): 16621, (('S1',), ('Ym',)): 16756, (('S1',), ('Yp',)): 16794, (('S2',), ('Zm',)): 33333, (('S2',), ('Xm',)): 16967, (('S2',), ('Xp',)): 16490, (('S2',), ('Ym',)): 17024, (('S2',), ('Yp',)): 16709, (('S3',), ('Zm',)): 33752, (('S3',), ('Xm',)): 16689, (('S3',), ('Xp',)): 16627, (('S3',), ('Ym',)): 16519, (('S3',), ('Yp',)): 16752, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '10': {(('S0',), ('Zm',)): 49758, (('S0',), ('Xm',)): 25049, (('S0',), ('Xp',)): 24925, (('S0',), ('Yp',)): 24518, (('S0',), ('Ym',)): 25163, (('S1',), ('Zm',)): 16745, (('S1',), ('Xm',)): 8280, (('S1',), ('Xp',)): 8184, (('S1',), ('Ym',)): 8332, (('S1',), ('Yp',)): 8309, (('S2',), ('Zm',)): 16695, (('S2',), ('Xm',)): 8293, (('S2',), ('Xp',)): 8248, (('S2',), ('Ym',)): 8276, (

{'11': {(('S0',), ('Zm',)): 2, (('S0',), ('Xp',)): 2, (('S1',), ('Ym',)): 1, (('S2',), ('Ym',)): 1, (('S0',), ('Zp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S2',), ('Zp',)): 0, (('S2',), ('Zm',)): 0, (('S2',), ('Xp',)): 0, (('S2',), ('Xm',)): 0, (('S2',), ('Yp',)): 0, (('S3',), ('Zp',)): 0, (('S3',), ('Zm',)): 0, (('S3',), ('Xp',)): 0, (('S3',), ('Xm',)): 0, (('S3',), ('Yp',)): 0, (('S3',), ('Ym',)): 0}, '10': {(('S0',), ('Zm',)): 99998, (('S0',), ('Xp',)): 50088, (('S0',), ('Xm',)): 49910, (('S0',), ('Yp',)): 49749, (('S0',), ('Ym',)): 50251, (('S1',), ('Zm',)): 33188, (('S1',), ('Xp',)): 16578, (('S1',), ('Xm',)): 16624, (('S1',), ('Ym',)): 16710, (('S1',), ('Yp',)): 16621, (('S2',), ('Zm',)): 33013, (('S2',), ('Xp',)): 16627, (('S2',), ('Xm',)): 16525, (('S2',), ('Ym',)): 16604, (('S2',), ('Yp',)): 16611, (('S3',), ('Zm',)): 33510, 

{'001': {(('S0',), ('Zp',)): 3, (('S0',), ('Xp',)): 2, (('S0',), ('Xm',)): 4, (('S0',), ('Yp',)): 8, (('S0',), ('Ym',)): 4, (('S1',), ('Zp',)): 2, (('S1',), ('Zm',)): 2, (('S1',), ('Xm',)): 2, (('S1',), ('Yp',)): 3, (('S1',), ('Ym',)): 3, (('S2',), ('Zp',)): 2, (('S2',), ('Zm',)): 5, (('S2',), ('Xp',)): 11, (('S2',), ('Xm',)): 1, (('S2',), ('Yp',)): 6, (('S3',), ('Zp',)): 3, (('S3',), ('Zm',)): 6, (('S3',), ('Xp',)): 8, (('S3',), ('Xm',)): 4, (('S3',), ('Ym',)): 5, (('S0',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S2',), ('Ym',)): 0, (('S3',), ('Yp',)): 0}, '101': {(('S0',), ('Zp',)): 4, (('S0',), ('Xm',)): 2, (('S0',), ('Xp',)): 3, (('S0',), ('Yp',)): 3, (('S0',), ('Ym',)): 3, (('S1',), ('Zp',)): 4, (('S1',), ('Zm',)): 5, (('S1',), ('Xp',)): 1, (('S1',), ('Xm',)): 12, (('S1',), ('Yp',)): 1, (('S1',), ('Ym',)): 2, (('S2',), ('Zp',)): 1, (('S2',), ('Zm',)): 7, (('S2',), ('Xm',)): 3, (('S2',), ('Xp',)): 11, (('S2',), ('Ym',)): 2, (('S2',), ('Yp',)): 14, (('S3',), ('Zm',)): 7, (('S3',), ('

{'10': {(('S0',), ('Zm',)): 100000, (('S0',), ('Xm',)): 49801, (('S0',), ('Xp',)): 50199, (('S0',), ('Ym',)): 50260, (('S0',), ('Yp',)): 49740, (('S1',), ('Zm',)): 33353, (('S1',), ('Xp',)): 16500, (('S1',), ('Xm',)): 16635, (('S1',), ('Yp',)): 16691, (('S1',), ('Ym',)): 16636, (('S2',), ('Zm',)): 33232, (('S2',), ('Xp',)): 16701, (('S2',), ('Xm',)): 16742, (('S2',), ('Ym',)): 16961, (('S2',), ('Yp',)): 16545, (('S3',), ('Zm',)): 33441, (('S3',), ('Xp',)): 16562, (('S3',), ('Xm',)): 16525, (('S3',), ('Yp',)): 16591, (('S3',), ('Ym',)): 16854, (('S0',), ('Zp',)): 0, (('S1',), ('Zp',)): 0, (('S2',), ('Zp',)): 0, (('S3',), ('Zp',)): 0}, '01': {(('S1',), ('Zm',)): 66647, (('S1',), ('Xm',)): 33322, (('S1',), ('Xp',)): 33543, (('S1',), ('Yp',)): 33455, (('S1',), ('Ym',)): 33217, (('S2',), ('Zm',)): 66768, (('S2',), ('Xm',)): 33352, (('S2',), ('Xp',)): 33205, (('S2',), ('Yp',)): 33363, (('S2',), ('Ym',)): 33129, (('S3',), ('Zm',)): 66559, (('S3',), ('Xp',)): 33490, (('S3',), ('Xm',)): 33423, 

In [309]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [310]:
direct_recombined_dist

{'10100': 2.1068626241999232e-05,
 '11100': -2.55017935529018e-06,
 '10110': 3.7924667378596354e-05,
 '11010': 7.898664853822038e-05,
 '11110': 1.5848063212702188e-05,
 '10010': 0.003948787959654314,
 '10000': 0.0004564091313715184,
 '11000': -2.6050552035206826e-05,
 '00100': -5.193181706392385e-09,
 '01100': 8.376847844198152e-11,
 '00110': -1.53223901050008e-07,
 '01010': -8.911597694153842e-08,
 '01110': -1.0090652206608841e-09,
 '00010': -8.868887442538398e-06,
 '00000': -1.447264190810277e-07,
 '01000': -1.9434369090142782e-10,
 '00101': 0.000273970046179156,
 '01101': 2.5070311804927164e-06,
 '00111': 0.01734947461786064,
 '01011': 0.009662759453497424,
 '01111': 0.00016708052146070467,
 '00011': 0.9521491808044593,
 '00001': 0.0156860945444481,
 '01001': 0.00017433029036898119,
 '10101': 5.156601728709749e-09,
 '11101': -6.0055508379156426e-12,
 '10111': 2.3499435207873277e-07,
 '11011': 1.2442097798468136e-07,
 '11111': 2.466758562858446e-09,
 '10011': 1.2856799874156307e-05,


In [311]:
likely_recombined_dist

{'10100': 6.405683116543741e-06,
 '10110': 9.958877306485323e-05,
 '11010': 0.00018245931571289885,
 '11110': 1.3856995996604352e-06,
 '10010': 0.008632836430271164,
 '10000': 0.0003799672443188104,
 '11000': 2.1259890748536475e-07,
 '00101': 0.00021790046069157575,
 '00111': 0.01712924396770827,
 '01011': 0.009508062988304701,
 '01111': 9.384743505691595e-05,
 '00011': 0.9481074759295954,
 '00001': 0.015560410953321743,
 '01001': 8.020252033006364e-05}

In [312]:
filtered_recombined_lists

[{'0111': 0.062408615092238384,
  '1111': 0.06004287860080369,
  '1001': 0.06230167497036156,
  '0001': 0.06391007396837373,
  '1011': 0.06046112001347265,
  '0011': 0.0625230962986704,
  '1101': 0.06124610258538095,
  '0101': 0.06328567544234998,
  '0110': 0.06234827692442673,
  '1110': 0.059991051465097645,
  '1000': 0.062269653003440416,
  '0000': 0.06386373797352271,
  '1010': 0.060399676744957054,
  '0010': 0.06246550218975069,
  '1100': 0.06120521389780737,
  '0100': 0.0632423592408941},
 {'1101': 2.2038025725300177e-05,
  '1011': 0.0023924649469073286,
  '0101': 0.004277934877226158,
  '1111': 1.8227258307263007e-05,
  '0111': 0.004356681254354672,
  '0011': 0.2398670955607841,
  '1001': 0.0023820671396971075,
  '0001': 0.24419040019431731,
  '1100': 2.1819813404715046e-05,
  '1010': 0.0023684027721147228,
  '0100': 0.004231651272069869,
  '1110': 1.8020183514687604e-05,
  '0110': 0.004313026558492125,
  '0010': 0.23727590103322782,
  '1000': 0.002357510170017693,
  '0000': 0.24

In [313]:
from utils.utils import filter_results

In [314]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [315]:
filter_direct_recombined

{'0100': -5.193181706392385e-09,
 '1100': 8.376847844198152e-11,
 '0110': -1.53223901050008e-07,
 '1010': -8.911597694153842e-08,
 '1110': -1.0090652206608841e-09,
 '0010': -8.868887442538398e-06,
 '0000': -1.447264190810277e-07,
 '1000': -1.9434369090142782e-10,
 '0101': 0.000273970046179156,
 '1101': 2.5070311804927164e-06,
 '0111': 0.01734947461786064,
 '1011': 0.009662759453497424,
 '1111': 0.00016708052146070467,
 '0011': 0.9521491808044593,
 '0001': 0.0156860945444481,
 '1001': 0.00017433029036898119}

In [316]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [317]:
filter_likely_recombined

{'0101': 0.00021790046069157575,
 '0111': 0.01712924396770827,
 '1011': 0.009508062988304701,
 '1111': 9.384743505691595e-05,
 '0011': 0.9481074759295954,
 '0001': 0.015560410953321743,
 '1001': 8.020252033006364e-05}

In [318]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [319]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [320]:
likely_recombined_dist['00011']

0.9481074759295954

In [321]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.957010405680093

In [322]:
norm_filter_dist

{'0101': 0.00021994659210957355,
 '0111': 0.017290091191884113,
 '1011': 0.009597345710988843,
 '1111': 9.472868232348443e-05,
 '0011': 0.957010405680093,
 '0001': 0.01570652650363999,
 '1001': 8.0955638961062e-05}

In [323]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(filtered_recombined_lists[i], shots = 1, Pauli = Pauli_tuple[0])  

In [324]:
final_expect_val

(-1.0347795650666873+0j)